In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../tftools/')

from tf_object import *

In [2]:
class UserRankModel(TFModel):
    def __init__(self, config, sess, current_task_name='user_rank'):
        super(UserRankModel, self).__init__(config, sess)
        #self.net_size = config.net_size
        self.feature_num = config.feature_num
        self.event_num = config.event_num
        self.current_task_name = current_task_name
        self.init_scale = config.init_scale
        
    def build_input(self):
        with tf.name_scope('input'):
            inputX = tf.placeholder(tf.float32, [None, self.feature_num], name="input_feature")
            eventID = tf.placeholder(tf.int32, [None, 1], name='event')
            inputLabel = tf.placeholder(tf.int32, [None], name='label')
            tf.add_to_collection(tf.GraphKeys.INPUTS, inputX)
            tf.add_to_collection(tf.GraphKeys.INPUTS, eventID)
            tf.add_to_collection(tf.GraphKeys.INPUTS, inputLabel)
            self.split_inputX = tf.split(inputX, self.gpu_num, 0)
            self.split_eventID = tf.split(eventID, self.gpu_num, 0)
            self.split_inputLabel = tf.split(inputLabel, self.gpu_num, 0)
        self.__build_global_setting__()
        with tf.name_scope('states_array'):
            self.last_flat = [[] for i in range(0,self.gpu_num)]
     
    def build_mlp_model(self, gpu_id=0, layer_num=6, net_size=256):
        with get_new_variable_scope('embedding') as embedding_scope:
            event_embedding = tf.reshape(my_embedding_layer(self.split_eventID[gpu_id], self.event_num, net_size, 
                                  layer_name='embedding_layer', init_scale=self.init_scale),[-1, net_size])
        dense = self.split_inputX[gpu_id]
        with get_new_variable_scope('mlp') as mlp_scope:
            for i in range(0, layer_num):            
                dense = my_full_connected(highway(dense), net_size, act=tf.nn.tanh)
        with tf.name_scope("dropout"):
            self.last_flat[gpu_id] = event_embedding + dense
            
    def build_prediction(self, gpu_id=0, num_classes=2, accK=1):
        prediction = my_full_connected(self.last_flat[gpu_id], num_classes)
        self.tower_prediction_results.append(tf.nn.softmax(prediction))
        self.params = tf.trainable_variables()[1:]
        with tf.name_scope('loss'): 
            loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.split_inputLabel[gpu_id], logits=prediction)
            grads, capped_gvs = my_compute_grad(self.opt, loss, self.params, 
                                                clip_type = 'clip_norm', 
                                                max_clip_grad=self.clip_gradients)           
        with tf.name_scope('accuracy'):
            accuracy = tf.to_float(tf.nn.in_top_k(prediction, self.split_inputLabel[gpu_id],k=accK))
        self.__add_to_tower_list__(grads,capped_gvs,loss,accuracy)
              
    def build_model(self, num_classes=2, *args, **kwargs):
        self.build_input()
        for idx, gpu_id in enumerate(self.gpus):
            with tf.device('/gpu:%d' % gpu_id):
                with tf.name_scope('Tower_%d' % (gpu_id)) as tower_scope:
                    gpu_scope = tf.variable_scope('gpu', reuse=(idx!=0))
                    with gpu_scope as gpu_scope:
                        self.build_mlp_model(gpu_id=idx, *args, **kwargs)
                        self.build_prediction(gpu_id=idx, num_classes=num_classes)                       
        self.build_model_aggregation()          

In [3]:
flags = tf.app.flags

flags.DEFINE_integer("feature_num", 52, "term number in input sequence(zero mask) [20001]")
flags.DEFINE_integer("event_num", 26, "the max length of input sequence [80]")
flags.DEFINE_float("init_scale", 1.0, "init scale for embedding layer")
flags.DEFINE_float("learning_rate", 0.01, "learning rate [0.001]")
flags.DEFINE_string("opt", 'sgd', "optimizer")
flags.DEFINE_integer("batch_size", 512, "batch size to use during training [128]")
flags.DEFINE_float("clip_gradients", 5.0, "clip gradients to this norm [5.0]")
flags.DEFINE_integer("n_epochs", 50, "number of epoch to use during training [10]")
flags.DEFINE_boolean("epoch_save", True, "save checkpoint or not in each epoch [True]")
flags.DEFINE_integer("print_step", 1000, "print step duraing training [100]")
flags.DEFINE_string("logs_dir", "logs/", "logs directory [logs/]")
flags.DEFINE_string("model_dir", "model/", "model directory [model/]")
flags.DEFINE_boolean("dir_clear", False, "clear the log and model directory")
flags.DEFINE_boolean("lr_annealing", True, "use lr annealing or not after each epoch [False]")
flags.DEFINE_integer("gpu_id", 0, "default gpu id [0]")
flags.DEFINE_integer("gpu_num", 4, "gpu_num")

FLAGS = flags.FLAGS

In [4]:
data = np.load('train_data.npy')
label = data[:,-2]
event = data[:,-1:]
data = data[:,:-2]
data = scale(data, axis=0, with_mean=True, with_std=True, copy=True )

In [5]:
idxs = np.append(np.random.choice(np.where(label==1)[0],300000), np.random.choice(np.where(label==0)[0],300000))
train_idxs, test_idxs = train_test_split(idxs, test_size=0.2, random_state=42)
test_idxs = np.sort(test_idxs)

In [6]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
with tf.Session(graph=graph_to_use, config=config) as session:
    #cnn_model = TextCNN(FLAGS, session, current_task_name='text_cnn_model')
    #cnn_model.build_model(num_classes=len(set(label[idxs])),max_conv_len=7, num_filters=512, dropout_keep_prob=0.5)
    user_rank_model = UserRankModel(FLAGS, session, current_task_name='user_rank_model')
    #label[idxs]
    user_rank_model.build_model(num_classes=2, net_size=512)
    user_rank_model.build_model_summary()
    display(user_rank_model.model_summary())
    user_rank_model.run([data,event,label], train_idxs, test_idxs)

Initializing


,variable_name,variable_shape,parameters
0,global/Variable:0,[],1
1,gpu/embedding/embedding_layer/embedding_table:0,"[26, 512]",13312
2,gpu/mlp/highway/output_lin_0/Matrix:0,"[52, 52]",2704
3,gpu/mlp/highway/transform_lin_0/Matrix:0,"[52, 52]",2704
4,gpu/mlp/fully_connected/W:0,"[52, 512]",26624
5,gpu/mlp/fully_connected/B:0,[512],512
6,gpu/mlp/highway_1/output_lin_0/Matrix:0,"[512, 512]",262144
7,gpu/mlp/highway_1/transform_lin_0/Matrix:0,"[512, 512]",262144
8,gpu/mlp/fully_connected_1/W:0,"[512, 512]",262144
9,gpu/mlp/fully_connected_1/B:0,[512],512


Epoch 1 ... training ...
epoch time: 0.38775242964426676
Epoch 1 training accuracy: 0.6003625
Epoch 1 ... test ...
Epoch 1 test accuracy: 0.609183333333
Model saved in file: model/user_rank_model.ckpt
{'loss': 0.66217916666666665, 'valid_los': 0.65487532552083338, 'best_accuracy': 0.60036250000000002, 'best_test_accuracy': 0.6091833333333333, 'epoch': 0, 'learning_rate': 0.01, 'valid_perplexity': 1.924902516197009}
Epoch 2 ... training ...
epoch time: 0.34612499475479125
Epoch 2 training accuracy: 0.611635416667
Epoch 2 ... test ...
Epoch 2 test accuracy: 0.608583333333
{'loss': 0.65325572916666663, 'valid_los': 0.65097981770833335, 'best_accuracy': 0.61163541666666665, 'best_test_accuracy': 0.6091833333333333, 'epoch': 1, 'learning_rate': 0.01, 'valid_perplexity': 1.9174186296401219}
Epoch 3 ... training ...
epoch time: 0.3486477176348368
Epoch 3 training accuracy: 0.6146125
Epoch 3 ... test ...
Epoch 3 test accuracy: 0.613216666667
Model saved in file: model/user_rank_model.ckpt
{'lo

epoch time: 0.3440946380297343
Epoch 21 training accuracy: 0.628710416667
Epoch 21 ... test ...
Epoch 21 test accuracy: 0.623
{'loss': 0.63695859374999997, 'valid_los': 0.64139980468750002, 'best_accuracy': 0.62871041666666672, 'best_test_accuracy': 0.62337500000000001, 'epoch': 20, 'learning_rate': 0.0005852766346593508, 'valid_perplexity': 1.8991374410295498}
Epoch 22 ... training ...
epoch time: 0.3457098364830017
Epoch 22 training accuracy: 0.62925625
Epoch 22 ... test ...
Epoch 22 test accuracy: 0.62325
{'loss': 0.6367449869791667, 'valid_los': 0.64115690104166667, 'best_accuracy': 0.62925624999999996, 'best_test_accuracy': 0.62337500000000001, 'epoch': 21, 'learning_rate': 0.0003901844231062339, 'valid_perplexity': 1.898676189643275}
Epoch 23 ... training ...
epoch time: 0.34747335116068523
Epoch 23 training accuracy: 0.628904166667
Epoch 23 ... test ...
Epoch 23 test accuracy: 0.6225
{'loss': 0.63671705729166672, 'valid_los': 0.64130898437499995, 'best_accuracy': 0.6292562499999